## 02. Creating a Major Merger (MM) catalogue to study AGN incidence due to galaxy mergers

The notebook is divided into the following sections:
1. Loading data and defining input parameters
2. Creating a Major Merger sample (MM) and finding pairs
2. Studying merger fraction $f_{MM}$ as a function of redshift

**Script written by**: Soumya Shreeram <br>
**Project supervised by**: Johan Comparat <br>
**Date**: 30th March 2021

Import required packages

In [1]:
# scipy modules
from scipy.spatial import cKDTree
from scipy.interpolate import interp1d

# astropy modules
import astropy.units as u
import astropy.io.fits as fits
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM, z_at_value

import numpy as np

# system imports
import os
import sys
import importlib as ib

# plotting imports
import matplotlib

Load the imported file(s) that contains all the functions used in this notebooks

In [2]:
%matplotlib inline
font = {'family' : 'serif',
        'weight' : 'medium',
        'size'   : 20}
matplotlib.rc('font', **font)

In [3]:
sys.path.append('imported_files/')
import Exploring_DM_Haloes as edh
import Agn_incidence_from_Major_Mergers as aimm
import plotting_aimm02 as pt

### 1. Defining input parameters

In [4]:
# look back into redshifts until...
redshift_limit = 0.2

# agn parameters
agn_FX_soft = 0

# galaxy parameters
galaxy_SMHMR_mass = 9 # unit: log 10, M_solar

# halo parameters
halo_mass_500c = 10**13.7 # solar masses
central_Mvir = 13.7 # unit: log 10, M_solar
cluster_params = [halo_mass_500c, central_Mvir]

# array deciding the cuts in the merger times and their linestyles
t_merger_cut_arr, l = [1, 2.5, 5], ['-', '--', ':']

# pixel number from the simulation file
pixel_no = '000000'

# number of cluster files for the chosen pixel_no
clu_files_no = 3

Define cosmology used in the notebook

In [5]:
cosmo = FlatLambdaCDM(H0=67.77*u.km/u.s/u.Mpc, Om0=0.307115)
h = 0.6777
L_box = 1000.0 / h

# interpolation function of z -> distance (depends on cosmology)
redshift_step = 0.0001
z_array = np.arange(0,redshift_limit+0.1, step=redshift_step)
f_z_to_comoving_dist = interp1d(z_array, cosmo.comoving_distance(z_array))

### 2. Open files and get relevant data

In [6]:
hd_agn, hd_halo, hd_clu = edh.getHeaders(pixel_no, np.array(['agn', 'halo', 'cluster']), clu_files_no=clu_files_no)

Extracting positions and redshifts of the AGNs, galaxies, and halos

In [7]:
# agns
pos_z_AGN, scale_merger_AGN, conditions_agn = edh.getAgnData(hd_agn, agn_FX_soft, redshift_limit)    

# galaxies and halos
pos_z_gal, scale_merger_gal, conditions_gal = edh.getGalaxyData(hd_halo, galaxy_SMHMR_mass, redshift_limit)
pos_z_halo, scale_merger_halo, conditions_halo = edh.getGalaxyData(hd_halo, '', redshift_limit)

print("AGNs: %d, Galaxies: %d, Halos: %d"%(len(pos_z_AGN[0]), len(pos_z_gal[0]), len(pos_z_halo[0]) ))

AGNs: 2295, Galaxies: 20438, Halos: 20438


Classifying the headers into central and satellite objects

In [8]:
cen_sat_halo = edh.cenSatObjects(conditions_halo, hd_halo, cluster_params[1])
cen_sat_AGN = edh.cenSatObjects(conditions_agn, hd_agn, cluster_params[1])

### 3.  Creating a Major Merger sample i.e. finding pairs

In [9]:
# ratio of masses to call a merger a major merger
mass_ratio_for_MM = 4

# time since merger [Gyr]
time_since_merger = 1

In [10]:
# generate mass bins for halos with M_1/M_2 < mass_ratio
mass_bins = aimm.mergerBins(hd_halo, galaxy_SMHMR_mass, mass_ratio=mass_ratio_for_MM)

# MM headers for halos ==> (1) given mass range and (2) given time since merger
hd_mm_halo, mm_conditions = aimm.createMMcatalog(hd_halo, conditions_halo, cosmo, mass_range=[mass_bins[0], mass_bins[1]], time_since_merger=time_since_merger)

In [ ]:
ib.reload(aimm)
pairs_halo_mm = aimm.findPairs(hd_mm_halo, leafsize=1000.0)

In [ ]:
pairs_halo_mm